#Aprendizaje no supervisado

El objetivo de este notebook es consolidar el conocimiento sobre los algoritmos de aprendizaje no supervisado revisados en clase. Se realizará experimentación con diversos algoritmos de clusterización y se utilizarán las métricas correspondientes para evaluar su desempeño. 

El alumno deberá completar las preguntas planetadas en el notebook como parte de la evaluación del curso.

## K-means

### Clusterización utilizando k-means

Ejemplo de clusterización en conjunto de datos iris. Se realiza el ajuste del modelo con diferentes valores de k. La gráfica de la inercia muestra cómo esta se reduce a medida que aumenta el valor de k.

In [0]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import datasets
from sklearn.cluster import KMeans

iris = datasets.load_iris()
X = iris.data
y = iris.target

inertias = []

for i in range(1, 11):
    kmeans = KMeans(n_clusters = i, max_iter = 300, n_init = 10, random_state = 42)
    kmeans.fit(X)
    inertias.append(kmeans.inertia_)
    
plt.plot(range(1, 11), inertias)
plt.title('Inercia por clusters')
plt.xlabel('# Clusters')
plt.ylabel('Inercia') 
plt.show()

A partir de k = 3 no aumenta mucho la inercia por lo que este es un valor adecuado para el número de clusters.

In [0]:
kmeans = KMeans(n_clusters = 3, max_iter = 300, n_init = 10, random_state = 0)
y_kmeans = kmeans.fit_predict(X)
print("Inercia: ", kmeans.inertia_)
print("Score: ",kmeans.score(X))

**Pregunta 1 (1 punto):** ¿Por qué el valor de score es negativo?

Visualizamos los clusters respecto a las dos primeras características.

In [0]:
plt.scatter(X[y_kmeans == 0, 0], X[y_kmeans == 0, 1], s = 10, c = 'red', label = 'Iris-setosa')
plt.scatter(X[y_kmeans == 1, 0], X[y_kmeans == 1, 1], s = 10, c = 'blue', label = 'Iris-versicolour')
plt.scatter(X[y_kmeans == 2, 0], X[y_kmeans == 2, 1], s = 10, c = 'green', label = 'Iris-virginica')

plt.scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:,1], s = 50, c = 'yellow', label = 'Centroids')

plt.legend()

**Pregunta 2 (1 punto):** Muestre la visualización para las otras 2 características (índices 2 y 3).

In [0]:
# Código pregunta 2

Comparemos la ejecución de Acelerated KMeans (algoritmo por defecto) con Kmeans full.

In [0]:
%timeit -n 50 KMeans(algorithm="elkan").fit(X)
%timeit -n 50 KMeans(algorithm="full").fit(X)

In [0]:
kmeans_per_k = [KMeans(n_clusters=k, random_state=42).fit(X)
                for k in range(1, 10)]
inertias = [model.inertia_ for model in kmeans_per_k]

La métrica Silhouette Score permite realizar una mejor evaluación del modelo.

In [0]:
from sklearn.metrics import silhouette_score
silhouette_scores = [silhouette_score(X, model.labels_)
                     for model in kmeans_per_k[1:]]


In [0]:
from sklearn.metrics import silhouette_samples
from matplotlib.ticker import FixedLocator, FixedFormatter

plt.figure(figsize=(11, 9))

for k in (2, 3, 4, 5):
    plt.subplot(2, 2, k - 1)
    
    y_pred = kmeans_per_k[k - 1].labels_
    silhouette_coefficients = silhouette_samples(X, y_pred)

    padding = len(X) // 30
    pos = padding
    ticks = []
    for i in range(k):
        coeffs = silhouette_coefficients[y_pred == i]
        coeffs.sort()

        color = mpl.cm.Spectral(i / k)
        plt.fill_betweenx(np.arange(pos, pos + len(coeffs)), 0, coeffs,
                          facecolor=color, edgecolor=color, alpha=0.7)
        ticks.append(pos + len(coeffs) // 2)
        pos += len(coeffs) + padding

    plt.gca().yaxis.set_major_locator(FixedLocator(ticks))
    plt.gca().yaxis.set_major_formatter(FixedFormatter(range(k)))
    if k in (2, 4):
        plt.ylabel("Cluster")
    
    if k in (4, 5):
        plt.gca().set_xticks([-0.1, 0, 0.2, 0.4, 0.6, 0.8, 1])
        plt.xlabel("Silhouette Coefficient")
    else:
        plt.tick_params(labelbottom=False)

    plt.axvline(x=silhouette_scores[k - 2], color="red", linestyle="--")
    plt.title("$k={}$".format(k), fontsize=16)

plt.show()

Revisemos el score de siluetas para otra distribución de datos:

Adaptado del ejemplo de score de siluetas de sklearn.

In [0]:
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np

# Generando data de ejemplo usando blobs
# El conjunto de datos tiene 3 clusters juntos y uno alejado
X, y = make_blobs(n_samples=500,
                  n_features=2,
                  centers=4,
                  cluster_std=1,
                  center_box=(-10.0, 10.0),
                  shuffle=True,
                  random_state=1)  

# Se explorará las soluciones para 2 a 6 clusters
range_n_clusters = [2, 3, 4, 5, 6]

for n_clusters in range_n_clusters:

    fig, (ax1, ax2) = plt.subplots(1, 2)
    fig.set_size_inches(18, 7)

    ax1.set_xlim([-0.1, 1])

    ax1.set_ylim([0, len(X) + (n_clusters + 1) * 10])

    # Se ejecuta el algoritmo KMeans para cada cluster y se genera las etiquetas
    clusterer = KMeans(n_clusters=n_clusters, random_state=10)
    cluster_labels = clusterer.fit_predict(X)

    # Se utiliza la función silhouette_score para obtener el score de silueta promedio
    silhouette_avg = silhouette_score(X, cluster_labels)
    print("Para n_clusters =", n_clusters,
          "El score de silueta promedio es :", silhouette_avg)

    # Se utiliza la función silhouette_samples para obtener el score de cada instancia
    sample_silhouette_values = silhouette_samples(X, cluster_labels)

    # Se genera el gráfico de comparación.
    y_lower = 10
    for i in range(n_clusters):
        ith_cluster_silhouette_values = \
            sample_silhouette_values[cluster_labels == i]

        ith_cluster_silhouette_values.sort()

        size_cluster_i = ith_cluster_silhouette_values.shape[0]
        y_upper = y_lower + size_cluster_i

        color = cm.nipy_spectral(float(i) / n_clusters)
        ax1.fill_betweenx(np.arange(y_lower, y_upper),
                          0, ith_cluster_silhouette_values,
                          facecolor=color, edgecolor=color, alpha=0.7)

        ax1.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))

        y_lower = y_upper + 10 

    ax1.set_title("Gráfico de siluetas para varios clusters")
    ax1.set_xlabel("Valor del coeficiente de siluetas")
    ax1.set_ylabel("Cluster")

    ax1.axvline(x=silhouette_avg, color="red", linestyle="--")

    ax1.set_yticks([]) 
    ax1.set_xticks([-0.1, 0, 0.2, 0.4, 0.6, 0.8, 1])

    colors = cm.nipy_spectral(cluster_labels.astype(float) / n_clusters)
    ax2.scatter(X[:, 0], X[:, 1], marker='.', s=30, lw=0, alpha=0.7,
                c=colors, edgecolor='k')

    centers = clusterer.cluster_centers_

    ax2.scatter(centers[:, 0], centers[:, 1], marker='o',
                c="white", alpha=1, s=200, edgecolor='k')

    for i, c in enumerate(centers):
        ax2.scatter(c[0], c[1], marker='$%d$' % i, alpha=1,
                    s=50, edgecolor='k')

    ax2.set_title("Visualización de los datos clusterizados")
    ax2.set_xlabel("Característica 1")
    ax2.set_ylabel("Característica 2")

    plt.suptitle(("Análisis de siluetas para clusterización usando KMeans en datos de prueba"
                  "con n_clusters = %d" % n_clusters),
                 fontsize=14, fontweight='bold')

plt.show()

### Minibatch KMeans

Se puede utilizar un objeto memmap para alimentar un dataset muy grande a la clase MiniBacthKMeans. Esta utiliza un archivo (my_mnist.data en este caso) para almeacenar la data en el disco.

In [0]:
from sklearn.cluster import MiniBatchKMeans
from sklearn.datasets import fetch_openml

mnist = fetch_openml('mnist_784', version=1)
mnist.target = mnist.target.astype(np.int64)

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    mnist["data"], mnist["target"], random_state=42)

filename = "my_mnist.data"
X_mm = np.memmap(filename, dtype='float32', mode='write', shape=X_train.shape)
X_mm[:] = X_train

Podemos comparar el tiempo de ejecución de MiniBatchKMeans contra KMeans.

In [0]:
minibatch_kmeans = MiniBatchKMeans(n_clusters=10, batch_size=10, random_state=42)
%timeit minibatch_kmeans.fit(X_mm)

In [0]:
minibatch_kmeans.score(X_mm)

**Advertencia:** las siguientes instrucciones pueden demorar unos minutos en ejecutar.

In [0]:
kmeans = KMeans(n_clusters = 10, random_state=42)
%timeit kmeans.fit(X_train)

In [0]:
kmeans.score(X_train)

### Compresión de imágenes usando KMeans

Se utilizará las imágenes de ejemplo de sklearn.datasets. Se normaliza los datos y se utiliza el algoritmo kmeans para agrupar cada pixel en 16 grupos.

In [0]:
from sklearn.datasets import load_sample_images

# Cargando la imagen de prueba
dataset = load_sample_images() 
A = dataset.images[0] 
A = np.asarray(A, dtype=float)

# Dividir entre 255 para que los valores estén en el rango 0 - 1
A = A / 255

# Tamaño de la imagen
img_size = A.shape

# Convertir la imagen en una matrix N x 3, donde N = número de pixels.
# Cada fila contiene los valores de Rojo, Verde y Azul
# Esto nos da la matriz del conjunto de datos X que usaremos con k-means
X = A.reshape((img_size[0] * img_size[1], 3))

# Parámetros para el algoritmo KMeans
K = 10
max_iter = 10
kmeans = KMeans(n_clusters = K, random_state=42, max_iter = max_iter)
kmeans.fit(X)

Se utiliza los centroides para reconstruir la imagen en base al color de los centroides de cada cluster.

In [0]:
X_recovered =  kmeans.cluster_centers_[kmeans.labels_]

# Transformar la imagen recuperada en sus dimensiones originales
X_recovered = X_recovered.reshape((img_size[0], img_size[1], 3))

# Mostrar la imagen original y la recuperada
f, (ax1, ax2) = plt.subplots(1, 2)
ax1.axis('off')
ax1.imshow(A)
ax1.set_title('Original')
ax2.axis('off')
ax2.imshow(X_recovered)
ax2.set_title('Comprimida, con %d colores' % K)


**Pregunta 3 (2 puntos):** Comprima la imágen de su preferencia. Elija un número de colores que permita preservar los detalles.

### Segmentacion de imagenes usando KMeans

Adaptado de: https://github.com/ageron/handson-ml2.

Se clusteriza la imagen para diferentes valores de K. En este caso, K es el número de segmentos deseado. Para separar la imagen de la flor del fondo basta con K = 2.

In [0]:
A = dataset.images[1] 

# Normalizar imágen
A = A / 255

X = A.reshape(-1, 3)

segmented_imgs = []
n_colors = (8, 6, 4, 2)

for n_clusters in n_colors:
    kmeans = KMeans(n_clusters=n_clusters, random_state=42).fit(X)
    segmented_img = kmeans.cluster_centers_[kmeans.labels_]
    segmented_imgs.append(segmented_img.reshape(A.shape))

plt.figure(figsize=(10,5))
plt.subplots_adjust(wspace=0.05, hspace=0.1)

plt.subplot(231)
plt.imshow(A)
plt.title("Original image")
plt.axis('off')

for idx, n_clusters in enumerate(n_colors):
    plt.subplot(232 + idx)
    plt.imshow(segmented_imgs[idx])
    plt.title("{} colors".format(n_clusters))
    plt.axis('off')

plt.show()

## DBSCAN

Adaptado de: https://github.com/ageron/handson-ml2


Utilizaremos el dataset moons de sklearn. Este contiene dos clases distribuidas en forma de lunas. Visualizamos el conjunto de datos.

In [0]:
from sklearn.datasets import make_moons
from sklearn.cluster import DBSCAN

X, y = make_moons(n_samples=1000, noise=0.05, random_state=42)

plt.scatter(X[y == 0, 0], X[y == 0, 1], s = 5, c = 'red', label = 'Clase 1')
plt.scatter(X[y == 1, 0], X[y == 1, 1], s = 5, c = 'blue', label = 'Clase 2')
plt.legend()
plt.show()

Entrenamos dos modelos DBSCAN en los datos.

In [0]:
dbscan = DBSCAN(eps=0.05, min_samples=5)
dbscan.fit(X)
np.unique(dbscan.labels_)

In [0]:
dbscan2 = DBSCAN(eps=0.2)
dbscan2.fit(X)
np.unique(dbscan2.labels_)

Graficamos el resultado de cada ejecución de DBSCAN:

In [0]:
def plot_dbscan(dbscan, X, size, show_xlabels=True, show_ylabels=True):
    core_mask = np.zeros_like(dbscan.labels_, dtype=bool)
    core_mask[dbscan.core_sample_indices_] = True
    anomalies_mask = dbscan.labels_ == -1
    non_core_mask = ~(core_mask | anomalies_mask)

    cores = dbscan.components_
    anomalies = X[anomalies_mask]
    non_cores = X[non_core_mask]
    
    plt.scatter(cores[:, 0], cores[:, 1],
                c=dbscan.labels_[core_mask], marker='o', s=size, cmap="Paired")
    plt.scatter(cores[:, 0], cores[:, 1], marker='*', s=20, c=dbscan.labels_[core_mask])
    plt.scatter(anomalies[:, 0], anomalies[:, 1],
                c="r", marker="x", s=100)
    plt.scatter(non_cores[:, 0], non_cores[:, 1], c=dbscan.labels_[non_core_mask], marker=".")
    if show_xlabels:
        plt.xlabel("$x_1$", fontsize=14)
    else:
        plt.tick_params(labelbottom=False)
    if show_ylabels:
        plt.ylabel("$x_2$", fontsize=14, rotation=0)
    else:
        plt.tick_params(labelleft=False)
    plt.title("eps={:.2f}, min_samples={}".format(dbscan.eps, dbscan.min_samples), fontsize=14)

In [0]:
plt.figure(figsize=(9, 3.2))

plt.subplot(121)
plot_dbscan(dbscan, X, size=100)

plt.subplot(122)
plot_dbscan(dbscan2, X, size=600, show_ylabels=False)

plt.show()

**Pregunta 4 (2 puntos):** ¿Qué diferencia hay entre los gráficos de las dos ejecuciones? Explique a que se deben estas diferencias.

## Birch

Se utilizará el algoritmo Birch para realizar la clusterización sobre un conjunto de datos artificial generado con 6 clusters. 

In [0]:
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
sns.set()
from sklearn.datasets.samples_generator import make_blobs
from sklearn.cluster import Birch

Generar el conjunto de datos y graficarlo, se pueden distinguir los 6 clusters.

In [0]:
X, clusters = make_blobs(n_samples=450, centers=6, cluster_std=0.70, random_state=0)
plt.scatter(X[:,0], X[:,1], alpha=0.7, edgecolors='b')

Se utiliza la clase Birch para construir el árbol.

In [0]:
brc = Birch(branching_factor=50, n_clusters=None, threshold=1.5)
brc.fit(X)

Con la función predict se puede obtener las etiquetas de cada instancia.

In [0]:
labels = brc.predict(X)
labels

In [0]:
plt.scatter(X[:,0], X[:,1], c=labels, cmap='rainbow', alpha=0.7, edgecolors='b')

Se puede utilizar la función transform para transformar un punto X en su distancia a cada cluster. Estos valores pueden ser utilizados como características adicionales para cada instancia.

In [0]:
X_transformed = brc.transform(X)
X_transformed

## Gaussian Mixture Models

Adaptado de: https://github.com/ageron/handson-ml2.

Exploraremos diferentes datasets con distribuciones de origen gaussiano para experimentar con el algoritmo GMM.

In [0]:
X1, y1 = make_blobs(n_samples=1000, centers=((4, -4), (0, 0)), random_state=42)
X1 = X1.dot(np.array([[0.374, 0.95], [0.732, 0.598]]))
X2, y2 = make_blobs(n_samples=250, centers=1, random_state=42)
X2 = X2 + [6, -8]
X = np.r_[X1, X2]
y = np.r_[y1, y2]

plt.plot(X[:, 0], X[:, 1], 'k.', markersize=2)
plt.show()

In [0]:
from sklearn.mixture import GaussianMixture
gm = GaussianMixture(n_components=3, n_init=10, random_state=42)
gm.fit(X)

Se puede utilizar el modelo para generar muestras nuevas de la distribución original.

In [0]:
X_new, y_new = gm.sample(100)
plt.plot(X_new[:, 0], X_new[:, 1], 'k.', markersize=2)
plt.show()

**Pregunta 5 (2 puntos):** ¿Qué particularidad se puede observar en el gráfico de las muestras generadas? Pruebe generar y graficar un mayor número de muestras.

Grafiquemos las fronteras de decisión y los contornos de densidad obtenidos por el modelo.

In [0]:
from matplotlib.colors import LogNorm

def plot_centroids(centroids, weights=None, circle_color='w', cross_color='k'):
    if weights is not None:
        centroids = centroids[weights > weights.max() / 10]
    plt.scatter(centroids[:, 0], centroids[:, 1],
                marker='o', s=30, linewidths=8,
                color=circle_color, zorder=10, alpha=0.9)
    plt.scatter(centroids[:, 0], centroids[:, 1],
                marker='x', s=50, linewidths=50,
                color=cross_color, zorder=11, alpha=1)
    
def plot_gaussian_mixture(clusterer, X, resolution=1000, show_ylabels=True):
    mins = X.min(axis=0) - 0.1
    maxs = X.max(axis=0) + 0.1
    xx, yy = np.meshgrid(np.linspace(mins[0], maxs[0], resolution),
                         np.linspace(mins[1], maxs[1], resolution))
    Z = -clusterer.score_samples(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)

    plt.contourf(xx, yy, Z,
                 norm=LogNorm(vmin=1.0, vmax=30.0),
                 levels=np.logspace(0, 2, 12))
    plt.contour(xx, yy, Z,
                norm=LogNorm(vmin=1.0, vmax=30.0),
                levels=np.logspace(0, 2, 12),
                linewidths=1, colors='k')

    Z = clusterer.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    plt.contour(xx, yy, Z,
                linewidths=2, colors='w', linestyles='dashed')
    
    plt.plot(X[:, 0], X[:, 1], 'k.', markersize=2)
    plot_centroids(clusterer.means_, clusterer.weights_)

    plt.xlabel("$x_1$", fontsize=14)
    if show_ylabels:
        plt.ylabel("$x_2$", fontsize=14, rotation=0)
    else:
        plt.tick_params(labelleft=False)
        
def compare_gaussian_mixtures(gm1, gm2, X):
    plt.figure(figsize=(9, 4))

    plt.subplot(121)
    plot_gaussian_mixture(gm1, X)
    plt.title('covariance_type="{}"'.format(gm1.covariance_type), fontsize=14)

    plt.subplot(122)
    plot_gaussian_mixture(gm2, X, show_ylabels=False)
    plt.title('covariance_type="{}"'.format(gm2.covariance_type), fontsize=14)

In [0]:
plt.figure(figsize=(8, 4))
plot_gaussian_mixture(gm, X)
plt.show()

Como se vió en clase, es posible restringir las matrices de covarianza usando el parámetro covariance_type:

* "full" (por defecto): sin restricción, los clusters pueden tomar cualquier forma elipsoidal.
* "tied": los clusters deben tener la misma forma que puede ser cualquier elipsoide.
* "spherical": los clusters deben ser esféricos, solo varía el diámetro.
* "diag": los clusters pueden tener cualquier forma elipsoidal pero los ejes de la elipse deben ser paralelos a los ejes.

In [0]:
gm_full = GaussianMixture(n_components=3, n_init=10, covariance_type="full", random_state=42)

gm_tied = GaussianMixture(n_components=3, n_init=10, covariance_type="tied", random_state=42)

gm_spherical = GaussianMixture(n_components=3, n_init=10, covariance_type="spherical", random_state=42)

gm_diag = GaussianMixture(n_components=3, n_init=10, covariance_type="diag", random_state=42)

gm_full.fit(X)
gm_tied.fit(X)
gm_spherical.fit(X)
gm_diag.fit(X)

In [0]:
compare_gaussian_mixtures(gm_tied, gm_spherical, X)
plt.show()

In [0]:
compare_gaussian_mixtures(gm_full, gm_diag, X)
plt.tight_layout()
plt.show()

### Detección de anomalías usando GMM


GMM puede usarse para detectar anomalías, basta explorar las instancias ubicadas en zonas de baja densidad. Se debe definir un límite de densidad para diferenciar una anomalía. Por ejemplo, si se conoce que 4% es la tasa de productos defectuosos en una fábrica, se puede usar como límite el valor de densidad que mantenga el 4% de instancias fuera.

In [0]:
# Cálculo del límite de densidad que permite descartar el 4% de las instancias.
densities = gm.score_samples(X)
density_threshold = np.percentile(densities, 4)
anomalies = X[densities < density_threshold]

Podemos imprimir las anomalías respecto a las fronteras de decisión y los contornos de densidad obtenidos por el modelo.

In [0]:
plt.figure(figsize=(8, 4))

plot_gaussian_mixture(gm, X)
plt.scatter(anomalies[:, 0], anomalies[:, 1], color='b', marker='x')
plt.ylim(top=5.1)

plt.show()

**Pregunta 6 (2 puntos):** ¿Cuántas instancias se utilizó? ¿Cuántas fueron detectadas como anomalías?

### GMM: Métricas de evaluación

Las métricas de evaluación revisadas para el algoritmo GMM son Bayesian information criterion (BIC) y Akaike information criterion (AIC). Se utilizarán estas métricas para detectar gráficamente el mejor valor de k para el conjunto de datos trabajado previamente. También se utilizará gridsearch para encontrar el mejor valor de k y covariance_type en base a la métrica BIC.

Entrenamiento de modelos GMM para k ∈ {1,2,3, ... ,11}

In [0]:
gms_per_k = [GaussianMixture(n_components=k, n_init=10, random_state=42).fit(X)
             for k in range(1, 11)]

Calculo de métricas BIC y AIC para cada modelo.

In [0]:
bics = [model.bic(X) for model in gms_per_k]
aics = [model.aic(X) for model in gms_per_k]

Gráfica de los indicadores BIC y AIC para cada valor de k.

In [0]:
plt.figure(figsize=(8, 3))
plt.plot(range(1, 11), bics, "bo-", label="BIC")
plt.plot(range(1, 11), aics, "go--", label="AIC")
plt.xlabel("$k$", fontsize=14)
plt.ylabel("Information Criterion", fontsize=14)
plt.axis([1, 9.5, np.min(aics) - 50, np.max(aics) + 50])
plt.annotate('Minimum',
             xy=(3, bics[2]),
             xytext=(0.35, 0.6),
             textcoords='figure fraction',
             fontsize=14,
             arrowprops=dict(facecolor='black', shrink=0.1)
            )
plt.legend()
plt.show()

Se utiliza gridsearch para determinar el mejor valor de k y covariance_type.

In [0]:
min_bic = np.infty

for k in range(1, 11):
    for covariance_type in ("full", "tied", "spherical", "diag"):
        bic = GaussianMixture(n_components=k, n_init=10,
                              covariance_type=covariance_type,
                              random_state=42).fit(X).bic(X)
        if bic < min_bic:
            min_bic = bic
            best_k = k
            best_covariance_type = covariance_type

In [0]:
best_k,best_covariance_type

**Pregunta 7 (2 puntos):** Replique el cálculo de mejores parámetros utilizando la métrica AIC.

### Bayesian Gaussian Mixture Model

En vez de realizar una búsqueda manual del número de clusters, se puede utilizar la clase BayesianGaussianMixture. Este modelo parte de un número inicial de componentes (que debe ser mayor al número de clusters) y asigna un peso de 0 a los componentes innecesarios. Podemos volver a experimentar con el conjunto de datos asumiendo que el número de componentes es menor a 10.

In [0]:
from sklearn.mixture import BayesianGaussianMixture

bgm = BayesianGaussianMixture(n_components=10, n_init=10, random_state=42)
bgm.fit(X)

Si revisamos los pesos del modelo, vemos que ha descartado 7 clusters, manteniendo solo 3.

In [0]:
np.round(bgm.weights_, 2)

Podemos graficar nuevamente las fronteras de decisión y los contornos de densidad obtenidos por el modelo.

In [0]:
plt.figure(figsize=(8, 5))
plot_gaussian_mixture(bgm, X)
plt.show()

## Comparación de modelos

Adaptado de los ejemplos de clusterización de Sklearn.

El ejemplo muestra las características de diferentes algoritmos de clusterización en conjuntos de datos 2D de diferentes formas. A excepción del último conjunto de datos, los parámetos de los conjuntos han sido configurados para producir clusters diferenciables. Algunos modelos son más sensibles a ciertas configuraciones que otros.

El último conjunto de datos es un ejemplo de datos homogeneos, no tiene solución como problema de clusterizacion. A pesar de que este experimento puede dar luces sobre el comportamiento de cada algoritmo, es posible que este no aplique para conjuntos de datos de dimensionalidad alta.

In [0]:
%matplotlib inline

print(__doc__)

import time
import warnings

import numpy as np
import matplotlib.pyplot as plt

from sklearn import cluster, datasets, mixture
from sklearn.neighbors import kneighbors_graph
from sklearn.preprocessing import StandardScaler
from itertools import cycle, islice

np.random.seed(0)

# ============
# Creación de datasets. Se crean conjuntos de datos de tamaño moderado, que permitan evaluar la 
# escalabilidad de los algoritmos sin elevar demasiado el tiempo de ejecución.
# ============
n_samples = 1500
noisy_circles = datasets.make_circles(n_samples=n_samples, factor=.5,
                                      noise=.05)
noisy_moons = datasets.make_moons(n_samples=n_samples, noise=.05)
blobs = datasets.make_blobs(n_samples=n_samples, random_state=8)
no_structure = np.random.rand(n_samples, 2), None

# Datos con distribución anisotropica 
random_state = 170
X, y = datasets.make_blobs(n_samples=n_samples, random_state=random_state)
transformation = [[0.6, -0.6], [-0.4, 0.8]]
X_aniso = np.dot(X, transformation)
aniso = (X_aniso, y)

# Datos con varianzas variadas
varied = datasets.make_blobs(n_samples=n_samples,
                             cluster_std=[1.0, 2.5, 0.5],
                             random_state=random_state)

# ============
# Parámetros de agrupamiento
# ============
plt.figure(figsize=(9 * 2 + 3, 12.5))
plt.subplots_adjust(left=.02, right=.98, bottom=.001, top=.96, wspace=.05,
                    hspace=.01)

plot_num = 1

default_base = {'quantile': .3,
                'eps': .3,
                'damping': .9,
                'preference': -200,
                'n_neighbors': 10,
                'n_clusters': 3,
                'min_samples': 20,
                'xi': 0.05,
                'min_cluster_size': 0.1}

datasets = [
    (noisy_circles, {'damping': .77, 'preference': -240,
                     'quantile': .2, 'n_clusters': 2,
                     'min_samples': 20, 'xi': 0.25}),
    (noisy_moons, {'damping': .75, 'preference': -220, 'n_clusters': 2}),
    (varied, {'eps': .18, 'n_neighbors': 2,
              'min_samples': 5, 'xi': 0.035, 'min_cluster_size': .2}),
    (aniso, {'eps': .15, 'n_neighbors': 2,
             'min_samples': 20, 'xi': 0.1, 'min_cluster_size': .2}),
    (blobs, {}),
    (no_structure, {})]

for i_dataset, (dataset, algo_params) in enumerate(datasets):
    # Actualización de parámetros para cada dataset
    params = default_base.copy()
    params.update(algo_params)

    X, y = dataset

    # Normalización de datos
    X = StandardScaler().fit_transform(X)

    # Estimacion del ancho de banda requerido por el algoritmo Meanshift
    bandwidth = cluster.estimate_bandwidth(X, quantile=params['quantile'])

    # Matriz de conectividad requerida por el algoritmo Ward
    connectivity = kneighbors_graph(
        X, n_neighbors=params['n_neighbors'], include_self=False)
    # la matriz de conectividad debe ser simétrica
    connectivity = 0.5 * (connectivity + connectivity.T)

    # ============
    # Creación de modelos de clusterización
    # ============
    
    # Meanshift
    ms = cluster.MeanShift(bandwidth=bandwidth, bin_seeding=True)
    
    # Mini Batch KMeans
    two_means = cluster.MiniBatchKMeans(n_clusters=params['n_clusters'])
    
    # Ward
    ward = cluster.AgglomerativeClustering(
        n_clusters=params['n_clusters'], linkage='ward',
        connectivity=connectivity)
    
    # Spectral Clustering
    spectral = cluster.SpectralClustering(
        n_clusters=params['n_clusters'], eigen_solver='arpack',
        affinity="nearest_neighbors")
    
    # DBSCAN
    dbscan = cluster.DBSCAN(eps=params['eps'])
    
    # OPTICS
    optics = cluster.OPTICS(min_samples=params['min_samples'],
                            xi=params['xi'],
                            min_cluster_size=params['min_cluster_size'])
    
    # Affinity Propagation
    affinity_propagation = cluster.AffinityPropagation(
        damping=params['damping'], preference=params['preference'])
    
    # Agglomerative Clustering
    average_linkage = cluster.AgglomerativeClustering(
        linkage="average", affinity="cityblock",
        n_clusters=params['n_clusters'], connectivity=connectivity)
    
    # Birch
    birch = cluster.Birch(n_clusters=params['n_clusters'])
    
    # Gaussian Mixture Model
    gmm = mixture.GaussianMixture(
        n_components=params['n_clusters'], covariance_type='full')

    clustering_algorithms = (
        ('MiniBatchKMeans', two_means),
        ('AffinityPropagation', affinity_propagation),
        ('MeanShift', ms),
        ('SpectralClustering', spectral),
        ('Ward', ward),
        ('Agglomerative Clustering', average_linkage),
        ('DBSCAN', dbscan),
        ('OPTICS', optics),
        ('Birch', birch),
        ('GaussianMixture', gmm)
    )

    for name, algorithm in clustering_algorithms:
        t0 = time.time()

        # Filtro de warnings
        with warnings.catch_warnings():
            warnings.filterwarnings(
                "ignore",
                message="the number of connected components of the " +
                "connectivity matrix is [0-9]{1,2}" +
                " > 1. Completing it to avoid stopping the tree early.",
                category=UserWarning)
            warnings.filterwarnings(
                "ignore",
                message="Graph is not fully connected, spectral embedding" +
                " may not work as expected.",
                category=UserWarning)
            algorithm.fit(X)

        t1 = time.time()
        if hasattr(algorithm, 'labels_'):
            y_pred = algorithm.labels_.astype(np.int)
        else:
            y_pred = algorithm.predict(X)

        plt.subplot(len(datasets), len(clustering_algorithms), plot_num)
        if i_dataset == 0:
            plt.title(name, size=18)

        colors = np.array(list(islice(cycle(['#377eb8', '#ff7f00', '#4daf4a',
                                             '#f781bf', '#a65628', '#984ea3',
                                             '#999999', '#e41a1c', '#dede00']),
                                      int(max(y_pred) + 1))))
        # Color negro para outliers, en caso existan
        colors = np.append(colors, ["#000000"])
        plt.scatter(X[:, 0], X[:, 1], s=10, color=colors[y_pred])

        plt.xlim(-2.5, 2.5)
        plt.ylim(-2.5, 2.5)
        plt.xticks(())
        plt.yticks(())
        plt.text(.99, .01, ('%.2fs' % (t1 - t0)).lstrip('0'),
                 transform=plt.gca().transAxes, size=15,
                 horizontalalignment='right')
        plot_num += 1

plt.show()

## Evaluación
**Pregunta 8 (8 puntos):** Elija uno de los conjuntos de datos de la carpeta Clase 7 en el repositorio del curso. Sigua los pasos para realizar el agrupamiento de un conjunto de datos desconocido.

1) Cargar el dataset seleccionado y explorar los datos.

In [0]:
# Parte 1

2) Exploración gráfica de los puntos para determinar los posibles clusters.

In [0]:
# Parte 2

3) Realizar cáculos o transformaciones adicionales si el algoritmo lo requiere (ej. normalización).

In [0]:
# Parte 3

4) Ejecutar alguno de los algoritmos de clusterización y estratégias estudiadas para encontrar el mejor número de clusters.



In [0]:
# Parte 4

5) Graficar el resultado de la cluterización.

In [0]:
# Parte 5